## Importing libraries and Loading data

In [1]:
import math, datetime, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
import itertools
import tqdm

def load_data(data_path):
    data = pd.read_csv(data_path)  
    return data

inbound = load_data("inbound_loads.csv")
outbound = load_data("outbound_laods.csv")
weather = load_data("weather.csv")
#For loop to ensure that all pallet data is in the same dataframe
pallet = load_data("Pallet_history_Gold_Spike[0].csv")
for x in range(1, 10):
    pallet = pd.concat([pallet, load_data(f"Pallet_history_Gold_Spike[{x}].csv")])
pallet = pallet.drop(['lot_code', 
                      'tran_type', 
                      'final_pallet_code', 
                      'warehouse_facility_id',
                      'source_system_id'], axis=1)
trainentest = load_data("demand_kWtrain_val.csv")
train = trainentest.iloc[:273988,:]
test = trainentest.iloc[273988:, :]

# Preprocessing the Data
### 1. Load features and get them into the base dataframe

- load all features into the base dataframe
- create base_df

### 2. Preprocessing of features
- create dummy
- normalization (DECISION: Min-Max Normalization) 
- one hot encoding
- interpolating (fill in in between values)

### 3. Cutting off the dataframe 
DECISION: ? 
Which datapoints to keep?
Pallet_movement_5min starts 2nd of Jan 2019, so first rows is NaN. 

-  Demand values we have:
12/31/2018  21:15:00  upandincluding 10/11/2021/ 6:07:00
-  Predicting the demand from 
10/11/2021/ 6:08 upandincluding 12/13/2021  17:59:00 

### 1. Load features and get them into the base dataframe
#### Load features

Total weight of pallets coming into the warehouse in the previous 1h, 5h, 10h, 23h

In [2]:
###Incoming weight feature preprocessing
#load all data
incoming_weight_1h = load_data('incoming_weight_df_1h.csv')
incoming_weight_5h = load_data('incoming_weight_df_5h.csv')
incoming_weight_10h = load_data('incoming_weight_df_10h.csv')
incoming_weight_23h = load_data('incoming_weight_df.csv')
#rename columns 
incoming_weight_5h = incoming_weight_5h.rename(columns = {'weight_23h' : 'weight_5h'})
incoming_weight_1h = incoming_weight_1h.rename(columns = {'weight_23h' : 'weight_1h'})
incoming_weight_10h = incoming_weight_10h.rename(columns = {'weight_23h' : 'weight_10h'})
#get them to a datetime object
incoming_weight_1h['datetime_local'] = pd.to_datetime(incoming_weight_1h['datetime_local'])
incoming_weight_5h['datetime_local'] = pd.to_datetime(incoming_weight_5h['datetime_local'])
incoming_weight_10h['datetime_local'] = pd.to_datetime(incoming_weight_10h['datetime_local'])
incoming_weight_23h['datetime_local'] = pd.to_datetime(incoming_weight_23h['datetime_local'])
#set index to be datetime
incoming_weight_1h.set_index('datetime_local', inplace=True)
incoming_weight_5h.set_index('datetime_local', inplace=True)
incoming_weight_10h.set_index('datetime_local', inplace=True)
incoming_weight_23h.set_index('datetime_local', inplace=True)
#reshape them to start at 2018-31-12 9:15PM
incoming_weight_1h = incoming_weight_1h[2361:]
incoming_weight_5h = incoming_weight_5h[2330:] 
incoming_weight_10h = incoming_weight_10h[2326:]
incoming_weight_23h = incoming_weight_23h[2323:]
#Drop duplicates
incoming_weight_1h = incoming_weight_1h.loc[~incoming_weight_1h.index.duplicated()]
incoming_weight_5h = incoming_weight_5h.loc[~incoming_weight_5h.index.duplicated()]
incoming_weight_10h = incoming_weight_10h.loc[~incoming_weight_10h.index.duplicated()]
incoming_weight_23h = incoming_weight_23h.loc[~incoming_weight_23h.index.duplicated()]


Total number of pallets moved within 5 minutes

In [3]:
#load data
pallet_move_5min = load_data('pallet_movement_5min_ft.csv')
#rename column
pallet_move_5min.rename(columns = {'quantity' : 'pallet_movement_5min'}, inplace = True)
#make index a datetime object and set as index
pallet_move_5min['datetime_local'] = pd.to_datetime(pallet_move_5min['datetime_local'])
pallet_move_5min.set_index('datetime_local', inplace = True)
#delete duplicates
pallet_move_5min = pallet_move_5min.loc[~pallet_move_5min.index.duplicated()]

In [4]:
base_door = load_data("feature_inbound_outbound_door_open.csv")
base_door = base_door.rename(columns = {'total' : 'doors_open'})
base_door['datetime_local'] = pd.to_datetime(base_door['datetime_local'])
base_door.set_index('datetime_local', inplace=True)

#### Create the base dataframe base_df

In [5]:
def addtimecol(df, colname): ####input df and colname 
    df[colname] = pd.to_datetime(df[colname])         
    df['year'] = df[colname].dt.year
    df['month'] = df[colname].dt.month
    df['weekday'] = df[colname].dt.weekday
    df['day'] = df[colname].dt.day
    df['hour'] = df[colname].dt.hour
    df['minute'] = df[colname].dt.minute        
    return df

#Create new dummy dfs
base_df = train.copy()
base_weather = weather.copy()

#Remove unnecessary columns
base_df = base_df.drop('Unnamed: 0', axis=1)
base_weather = base_weather.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
base_weather["localstrptime"]= pd.to_datetime(base_weather["localstrptime"])
base_df['datetime_local'] = pd.to_datetime(base_df['datetime_local'])
base_weather = base_weather.rename(columns={'localstrptime':'datetime_local'})

#Add time columns.
addtimecol(base_df, 'datetime_local')

#Set index to datetime
base_df.set_index('datetime_local', inplace=True)
base_weather.set_index('datetime_local', inplace=True)

#Concatenate the weather DataFrame to the base DataFrame
base_df = pd.concat([base_df, base_weather], axis=1)

#Concatenate the incoming weight dataframe with the base dataframe
base_df = pd.concat([base_df, incoming_weight_1h], axis=1)
base_df = pd.concat([base_df, incoming_weight_5h], axis=1)
base_df = pd.concat([base_df, incoming_weight_10h], axis=1)
base_df = pd.concat([base_df, incoming_weight_23h], axis=1)

#Concatenate the pallet movement feature
base_df = pd.concat([base_df, pallet_move_5min], axis = 1)

#Concatenate the door feature
base_df = pd.concat([base_df, base_door], axis=1)

### 2. Preprocessing of features

#### Creating the dummy df with which we will do model selection




In [6]:
dummy_df = base_df.copy()
dummy_df = dummy_df.reset_index()
dummy_df = dummy_df.drop(['hour'], axis=1)
dummy_df = dummy_df.drop(['datetime'], axis=1)
dummy_df = dummy_df.drop(['datetime_local'], axis=1)
dummy_df = dummy_df.drop(['datetime_UTC'], axis=1)
dummy_df

dummy_normalized_df = dummy_df.copy()
#Still drop demand_kW NaNs
dummy_normalized_df = dummy_normalized_df.dropna(subset=['demand_kW'], axis=0)

#### Preprocessing functions defined
DECISION: MinMax Normalization

In [7]:
def normalize_column(df, columnname):
    """Function which returns a Dataframe where the given column is normalized through min-max normalization."""
    df[f'{columnname}_normalized'] = (df[columnname] - df[columnname].min()) / (df[columnname].max() - df[columnname].min())
    return df.drop([columnname], axis=1)

def add_one_hot_encoder(df, colname):
    """
    Function which returns a DataFrame where the given column has been removed and replaced by
    one-hot-encoding columns for each value in the original column.
    """
    onehot = pd.get_dummies(df[colname], prefix=colname)
    return df.drop(colname, axis=1).join(onehot)

def interpolate_column(df, colname):
    df[f'{colname}_interpolated'] = df[colname].interpolate(method='linear')
    return df.drop([colname], axis=1)

#### interpolate/one-hot-encoding/normalizing
DECISION: ?  normalization of the doors_open

In [8]:
interpolate = ['Temperature', 'Relative Humidity']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated', 'weight_23h','weight_10h', 'weight_5h', 'weight_1h', 'pallet_movement_5min', 'doors_open']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    dummy_normalized_df = interpolate_column(dummy_normalized_df, x)
for x in to_normalize:
    dummy_normalized_df = normalize_column(dummy_normalized_df, x)
for x in add_one_hot_encoding:
    dummy_normalized_df = add_one_hot_encoder(dummy_normalized_df, x)

### 3. Create timeseries dummy + method for adding lags of a cycle we want to model
##### timeseries_dummy only consists of the range of dates we have for which we have information per minute, because timeseries need to have regular intervals. 


In [9]:
### create timeseries_dummy 
starttime = trainentest['datetime_local'].at[79678]
startime = pd.to_datetime(starttime)
onlyminutesstep1 = base_df[:1460735]
onlyminutes = onlyminutesstep1.loc[starttime:]
timeseries_dummy = onlyminutes.copy()

### Create rows for missing minutes
timeseries_dummy = timeseries_dummy.asfreq(freq = 'T', method = 'pad')

### about 6900 random rows don't have demand, ffill is fill them with the previous amount
timeseries_dummy['demand_kW'] = timeseries_dummy.demand_kW.fillna(method = 'ffill', axis = 0)

### edit timeseries_dummy just as dummy_dataset
timeseries_dummy.reset_index(inplace = True)
timeseries_dummy.drop(['hour'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime_local'], axis=1, inplace = True)
timeseries_dummy.drop(['datetime_UTC'], axis=1, inplace = True)

### setup a normalized dataframe
timeseries_normalized_dummy = timeseries_dummy.copy()

#### A method to create 'lags' of a series; aka previous values in a cycle.  

In [10]:
#These values are added as columns to the timeseries_dummy
#The amount of columns that are added are the amount of lags
#The column used is in the list [col_names]
#The period of which the previous timepoint is taken are the amount of minutes before it, so cycle of a day is 60x24 minutes

def add_lags_of_column(df, col_names, lag = 3, period = 1):
    new_dict={}
    for col_name in col_names:
        new_dict[col_name]=df[col_name]
        # create lagged Series
        for l in range(1,lag+1):
            new_dict['%s_%amin_%d' %(col_name, period,l)]=df[col_name].shift(l*period)
    return pd.DataFrame(new_dict,index=df.index)

##### Using method above to add a daycycle

In [11]:
add_daycycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 1440)
#add_hourcycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 60)
#add_weekcycle = add_lags_of_column(timeseries_dummy, ['demand_kW'], 10, 10080)

pd.concat([timeseries_dummy, add_daycycle], axis = 1)

,demand_kW,year,month,weekday,day,minute,Relative Humidity,Temperature,weight_1h,weight_5h,...,demand_kW_1440min_1,demand_kW_1440min_2,demand_kW_1440min_3,demand_kW_1440min_4,demand_kW_1440min_5,demand_kW_1440min_6,demand_kW_1440min_7,demand_kW_1440min_8,demand_kW_1440min_9,demand_kW_1440min_10
0,1334.544,2021.0,5.0,0.0,24.0,4.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1302.755,2021.0,5.0,0.0,24.0,5.0,100.00,71.6,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1407.071,2021.0,5.0,0.0,24.0,6.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1328.769,2021.0,5.0,0.0,24.0,7.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1413.847,2021.0,5.0,0.0,24.0,8.0,NaN,NaN,88553.0,478724.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201239,2217.847,2021.0,10.0,0.0,11.0,3.0,NaN,NaN,26159.0,333431.0,...,2083.348,2454.500,2664.214,2576.234,2320.085,2490.769,2404.005,2366.596,2654.723,2358.351
201240,2184.012,2021.0,10.0,0.0,11.0,4.0,NaN,NaN,26159.0,333431.0,...,2089.805,2452.624,2653.190,2560.751,2191.568,2458.903,2413.946,2393.992,2697.868,2382.798
201241,2159.482,2021.0,10.0,0.0,11.0,5.0,87.75,57.2,26159.0,318331.0,...,2132.082,2497.049,2637.603,2549.844,2130.947,2412.434,2431.350,2426.633,2723.577,2394.729
201242,2145.155,2021.0,10.0,0.0,11.0,6.0,NaN,NaN,26159.0,277345.0,...,2135.021,2587.285,2648.607,2554.448,2102.054,2408.541,2429.524,2472.633,2737.067,2392.788


#### Normalize timeseries_dummy

In [12]:
interpolate = ['Temperature', 'Relative Humidity']
to_normalize = ['Relative Humidity_interpolated', 'Temperature_interpolated', 'weight_23h','weight_10h', 'weight_5h', 'weight_1h', 'pallet_movement_5min', 'doors_open']
add_one_hot_encoding = ['weekday', 'year', 'month']


for x in interpolate:
    timeseries_normalized_dummy = interpolate_column(timeseries_normalized_dummy, x)
for x in to_normalize:
    timeseries_normalized_dummy = normalize_column(timeseries_normalized_dummy, x)
for x in add_one_hot_encoding:
    timeseries_normalized_dummy = add_one_hot_encoder(timeseries_normalized_dummy, x)

# Model selection and building
## 1. Importing model libraries

## 2. Divide Train and Test data

## 3. Test models 

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

from sklearn.ensemble import RandomForestRegressor

## 2. Dividing train & test data

In [14]:
def test_model(model, data, debug=False):
    xtrain, ytrain, xtest, ytest = data
    if debug:
        print("Fitting model...")
    model.fit(xtrain, ytrain)
    if debug:
        print('Predicting...')
    acc = model.predict(xtest)
    if debug:
        print('Calculating mean absolute error...')
    return mean_absolute_error(list(ytest), acc)


##### dummy_normalized_df['pallet_movement_5min'] drop NaN values

In [15]:
dummy_normalized_df = dummy_normalized_df.dropna(subset = ['pallet_movement_5min_normalized'])

In [16]:
#Define train, test sets\n",
train, test = train_test_split(dummy_normalized_df, shuffle=True)
X_train = train.copy().drop(['demand_kW'], axis=1)
Y_train = train['demand_kW']
X_test = test.copy().drop(['demand_kW'], axis=1)
Y_test = test['demand_kW']

# Model Testing

### Use the find_optimal_parameters function imported from model_features.py



In [17]:
parameter_settings = {
    'n_estimators': [10, 20],
    #'criterion': ['squared_error', 'friedman_mse', 'squared_error', 'poisson'],
    #'max_depth': [2, 4, 6],
    #'min_samples_split': [2, 4, 8]
}
data = [X_train, Y_train, X_test, Y_test]
model = RandomForestRegressor

In [ ]:
from model_features import read_model
from model_features import find_optimal_parameters
from model_features import run_timeseries_model

find_optimal_parameters(dummy_normalized_df, model, parameter_settings, n_splits=2, verbose=True)

Testing model Model 1 with parameters: {'n_estimators': 10}
Starting fold 1
Training model...
Predicting...
Calculating rmse's...
Fold 1 train error: 45.93951387187146. Test error: 656.6920876372201. Time taken: 5.792354345321655 s.
Starting fold 2
Training model...
Predicting...
Calculating rmse's...
Fold 2 train error: 42.8272329452065. Test error: 529.2718317333357. Time taken: 11.185985088348389 s.
Finished testing model. Test RMSE: 529.2718317333357. Time taken: 16.978339433670044.
Testing model Model 2 with parameters: {'n_estimators': 20}
Starting fold 1
Training model...
